# Can Training on Filtered Data Predict Real Data?

We're going to train EfficientNet on all of our filtered images and have it try to predict on the real images just to see what happens

## Imports

In [1]:
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.applications import EfficientNetB0


## Loading Data

In [2]:
test_data = np.load('CIFAKE_Test.npz')

train_color_balance = np.load('Color_Balenced_Train.npz')

train_gaussian_blur = np.load('Gaussian_Blur_Train.npz')

train_median_blur = np.load('Median_Blur_Train.npz')


In [3]:
X_test = test_data['images']
y_test = test_data['labels']

X_train_cb = train_color_balance['images']
y_train_cb = train_color_balance['labels']

X_train_gb = train_gaussian_blur['images']
y_train_gb = train_gaussian_blur['labels']

X_train_mb = train_median_blur['images']
y_train_mb = train_median_blur['labels']

label_names = test_data['label_names']


## Building Models

We'll define a function to make an EfficientNet model for binary classification so we don't have to duplicate the code

In [5]:
def make_eff_net(input_shape=(32, 32, 3)):
  efficient_net = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
  x = Dense(512, activation='relu')(efficient_net.output)
  x2 = Dense(256, activation='relu')(x)
  Ph = Dense(2, activation='softmax')(x2)
  return Model(inputs=efficient_net.inputs, outputs=Ph)


In [6]:
eff_net_cb = make_eff_net()
eff_net_gb = make_eff_net()
eff_net_mb = make_eff_net()


# Training the Models

We'll specify hyperparameters here to keep them the same for each model

In [7]:
batch_size = 100
epochs = 10
learning_rate = 2e-4
optimizer = Adam(learning_rate=learning_rate)
loss = 'sparse_categorical_crossentropy'

checkpt = ModelCheckpoint('Filtered_Images.h5', save_best_only=True, verbose=0)
tb = TensorBoard(log_dir='tb_logs')
e_stop = EarlyStopping(patience=3)


In [8]:
eff_net_cb.compile(optimizer=Adam(learning_rate=learning_rate), loss=loss, metrics=['accuracy'])
eff_net_gb.compile(optimizer=Adam(learning_rate=learning_rate), loss=loss, metrics=['accuracy'])
eff_net_mb.compile(optimizer=Adam(learning_rate=learning_rate), loss=loss, metrics=['accuracy'])


In [9]:
eff_net_cb_hist = eff_net_cb.fit(X_train_cb, y_train_cb, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[checkpt, tb, e_stop])


Epoch 1/10
800/800 [==============================] - ETA: 0s - loss: 0.4367 - accuracy: 0.5005

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


800/800 [==============================] - 92s 51ms/step - loss: 0.4367 - accuracy: 0.5005 - val_loss: 0.7461 - val_accuracy: 0.5011
Epoch 2/10
800/800 [==============================] - 38s 47ms/step - loss: 0.2850 - accuracy: 0.5012 - val_loss: 0.7947 - val_accuracy: 0.4996
Epoch 3/10
800/800 [==============================] - 36s 45ms/step - loss: 0.2304 - accuracy: 0.5018 - val_loss: 0.7243 - val_accuracy: 0.5014
Epoch 4/10
800/800 [==============================] - 37s 47ms/step - loss: 0.1929 - accuracy: 0.5020 - val_loss: 1.1592 - val_accuracy: 0.4996
Epoch 5/10
800/800 [==============================] - 38s 47ms/step - loss: 0.1631 - accuracy: 0.5023 - val_loss: 1.3464 - val_accuracy: 0.4998
Epoch 6/10
800/800 [==============================] - 40s 50ms/step - loss: 0.1377 - accuracy: 0.5026 - val_loss: 0.7084 - val_accuracy: 0.5010
Epoch 7/10
800/800 [==============================] - 39s 48ms/step - loss: 0.1182 - accuracy: 0.5029 - val_loss: 0.8112 - val_accuracy: 0.5020
Epo

In [10]:
cb_test_loss, cb_test_accuracy = eff_net_cb.evaluate(X_test, y_test)


625/625 [==============================] - 9s 11ms/step - loss: 442.6338 - accuracy: 0.4998


In [11]:
eff_net_gb_hist = eff_net_gb.fit(X_train_gb, y_train_gb, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[checkpt, tb, e_stop])


Epoch 1/10
800/800 [==============================] - 76s 53ms/step - loss: 0.3274 - accuracy: 0.5011 - val_loss: 0.2037 - val_accuracy: 0.5036
Epoch 2/10
800/800 [==============================] - 38s 47ms/step - loss: 0.1880 - accuracy: 0.5019 - val_loss: 0.1605 - val_accuracy: 0.5040
Epoch 3/10
800/800 [==============================] - 39s 49ms/step - loss: 0.1424 - accuracy: 0.5022 - val_loss: 0.1479 - val_accuracy: 0.5042
Epoch 4/10
800/800 [==============================] - 38s 47ms/step - loss: 0.1150 - accuracy: 0.5031 - val_loss: 0.1441 - val_accuracy: 0.5041
Epoch 5/10
800/800 [==============================] - 36s 46ms/step - loss: 0.0937 - accuracy: 0.5031 - val_loss: 0.1363 - val_accuracy: 0.5041
Epoch 6/10
800/800 [==============================] - 37s 47ms/step - loss: 0.0763 - accuracy: 0.5032 - val_loss: 0.1384 - val_accuracy: 0.5040
Epoch 7/10
800/800 [==============================] - 40s 51ms/step - loss: 0.0626 - accuracy: 0.5034 - val_loss: 0.1395 - val_accuracy:

In [12]:
gb_test_loss, gb_test_accuracy = eff_net_gb.evaluate(X_test, y_test)


625/625 [==============================] - 9s 12ms/step - loss: 4.9146 - accuracy: 0.5009


In [13]:
eff_net_mb_hist = eff_net_mb.fit(X_train_cb, y_train_cb, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[checkpt, tb, e_stop])


Epoch 1/10
800/800 [==============================] - 72s 50ms/step - loss: 0.4465 - accuracy: 0.5004 - val_loss: 1.0281 - val_accuracy: 0.4997
Epoch 2/10
800/800 [==============================] - 39s 48ms/step - loss: 0.2940 - accuracy: 0.5012 - val_loss: 0.9137 - val_accuracy: 0.5001
Epoch 3/10
800/800 [==============================] - 40s 50ms/step - loss: 0.2326 - accuracy: 0.5017 - val_loss: 0.7803 - val_accuracy: 0.5013
Epoch 4/10
800/800 [==============================] - 37s 47ms/step - loss: 0.1935 - accuracy: 0.5019 - val_loss: 0.4915 - val_accuracy: 0.5027
Epoch 5/10
800/800 [==============================] - 38s 47ms/step - loss: 0.1638 - accuracy: 0.5022 - val_loss: 0.8145 - val_accuracy: 0.5011
Epoch 6/10
800/800 [==============================] - 37s 46ms/step - loss: 0.1379 - accuracy: 0.5026 - val_loss: 1.3378 - val_accuracy: 0.4998
Epoch 7/10
800/800 [==============================] - 38s 47ms/step - loss: 0.1174 - accuracy: 0.5029 - val_loss: 0.8007 - val_accuracy:

In [14]:
mb_test_loss, mb_test_accuracy = eff_net_mb.evaluate(X_test, y_test)


625/625 [==============================] - 12s 15ms/step - loss: 209.4467 - accuracy: 0.5012


All of our models did very bad, scoring about 50% which is our baseline accuracy rate. This suggests that we should not be training on filtered images to predict real images, we need to filter the test images if we train on filtered images